# 📘 LangChain 文件問答系統實作教學（含 PDF/純文字檔）
本教學將帶你從零開始建立一個 LangChain 文件問答系統，透過文字或 PDF 文件建立知識庫，並使用語言模型進行問答。

### 🔧 安裝所需套件

In [22]:
!pip install -U langchain-community
!pip install pypdf
!pip install wikipedia
!pip install langchain_text_splitters
!pip install langchain_openai
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.1 MB/s eta 0:00:00


### 📂 載入本地文字檔（TextLoader）
* TextLoader 可將純文字檔讀取為 LangChain 文件物件。

* 若出現編碼錯誤，請明確指定 encoding='utf-8'。

In [4]:
from langchain_community.document_loaders import TextLoader

In [5]:
loader=TextLoader("demo.txt",encoding="utf-8")
docs=loader.load()
docs

[Document(metadata={'source': 'demo.txt'}, page_content='凡爾賽宮（法語：Château de Versailles）位於法國巴黎西南郊外伊夫林省的省會凡爾賽，1682年由太陽王路易十四開始建造，分多階段完工，在此期間的凡爾賽宮超越了巴黎，成為了法國實際意義上的首都。\n\n凡爾賽宮有著寢宮、花園、美術收藏庫、辯論場、劇場、情報中心和政治會議室等諸多功能，是一座兼具觀賞性和實用性的宮殿，以金色、藍色和粉橘色為主基調，傾其所能表現「絢爛豪華的奢侈美」，這種風格被後世稱為「巴洛克」風格，不過在其後任法國國王路易十五的末期和整個路易十六時期，還加入了一些甜美風格「洛可可」裝飾。到了20世紀以後，法國因為進入共和政府，皇室被廢除，於是改建為博物館對公眾開放，一躍成為法國著名景點和文化遺產。[1]並在1979年被聯合國教科文組織列為世界文化遺產。[2]\n\n凡爾賽宮的建築風格因為極盡奢華，在17～18世紀的歐洲各國引發巨大轟動，神聖羅馬帝國（現奧地利和德國）、俄羅斯帝國、波蘭立陶宛聯邦和瑞典王國等歐洲大國的君主紛紛倣效，形成一股修建皇宮的跟風之潮。\n\n興建\n\n凡爾賽宮所在地區原來是一片森林和沼澤荒地。1624年，法國國王路易十三以1萬里弗爾的價格買下117法畝荒地，在這裡修建一座二層的紅磚樓房，用作狩獵行宮。[3][4]二樓有國王辦公室、寢室、接見室、藏衣室、隨從人員臥室等房間，一層為家具儲藏室和兵器庫。當時的行宮擁有26個房間，如今擁有2300個房間、67個樓梯和5210件家具[5][6]。\n\n1660年，法國國王路易十四參觀財政大臣富凱的沃子爵城堡[7]，為其房屋與花園的宏偉壯麗所折服，當時王室在巴黎郊外的行宮——聖日耳曼宮、萬塞訥宮、聖克盧宮等無一可以與其相比。路易十四憤怒他不盡職守之餘將富凱以「貪污」罪名投入巴士底獄，並命令沃子爵府邸的兩位主要設計師：園林家安德烈·勒諾特爾和建築家路易·勒沃為其設計新的行宮。[8][9][10]16至17世紀期間巴黎市民不斷發生暴動，更是在1648年至1653年，發生兩次規模巨大的投石黨叛亂，導致路易十四決定將王室宮廷（當時位於巴黎城內的羅浮宮和杜伊勒里宮）遷出混亂喧鬧的巴黎城。[11][12]經過考察和權衡，他決定以路易十三在凡爾賽的狩獵行宮為基礎建造新宮殿。為此

In [3]:
# 檢查當前目錄有哪些檔案可讀取：
import os
print(os.listdir())

['.config', 'demo.txt', 'demo2.txt', 'sample_data']


### 📄 載入 PDF 文件（PyPDFLoader）
* 每一頁會被拆分為一份獨立的 Document，後續可用於建立知識向量庫。

In [6]:
from langchain_community.document_loaders import PyPDFLoader

In [7]:
loader=PyPDFLoader("論文介紹.pdf")
docs=loader.load()
docs

[Document(metadata={'producer': 'Skia/PDF m119', 'creator': 'Chromium', 'creationdate': '2024-04-02T04:28:01+00:00', 'moddate': '2024-04-02T04:28:01+00:00', 'source': '論文介紹.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='学 习 ⼤ 语 ⾔ 模 型 原 理 必 看 的  10 篇 论 ⽂\n1\n🚀\n学习⼤语⾔模型原理必看的  10 篇论\n⽂\n1.Transformer\nChatGPT 使⽤的预训练模型  GPT ，是在  Transformer 中的  decoder 基础上进⾏改造\n的。\n论⽂标题：Attention Is All You Need\n论⽂链接：https://arxiv.org/pdf/1706.03762.pdf\n摘要：占主导地位的序列转导模型是基于复杂的递归或卷积神经⽹络，包括⼀个编码器和\n⼀个解码器。性能最好的模型还通过注意机制将编码器和解码器连接起来。我们提出了⼀\n个新的简单的⽹络结构 –Transformer ，它只基于注意⼒机制，完全不需要递归和卷积。\n在两个机器翻译任务上的实验表明，这些模型在质量上更胜⼀筹，同时也更容易并⾏化，\n需要的训练时间也⼤⼤减少。我们的模型在 WMT 2014 英德翻译任务中达到了 28.4 \nBLEU ，⽐现有的最佳结果（包括合集）提⾼了 2 BLEU 以上。在 WMT 2014 英法翻译任\n务中，我们的模型在 8 个 GPU 上训练了 3.5 天后，建⽴了新的单模型最先进的 BLEU 得分，\n即 41.0 分，这只是⽂献中最佳模型的训练成本的⼀⼩部分。\n2.GPT-3\nGPT 家族与  BERT 模型都是知名的  NLP 预训练模型，都基于  Transformer 技术。 GPT\ue088\n1 只有 12 个  Transformer 层，⽽到了  GPT\ue0883 ，则增加到  96 层。\n论⽂标题：Language Models are Few-Shot Learners\n论⽂链接：https:

In [8]:
print(docs[0].page_content)

学 习 ⼤ 语 ⾔ 模 型 原 理 必 看 的  10 篇 论 ⽂
1
🚀
学习⼤语⾔模型原理必看的  10 篇论
⽂
1.Transformer
ChatGPT 使⽤的预训练模型  GPT ，是在  Transformer 中的  decoder 基础上进⾏改造
的。
论⽂标题：Attention Is All You Need
论⽂链接：https://arxiv.org/pdf/1706.03762.pdf
摘要：占主导地位的序列转导模型是基于复杂的递归或卷积神经⽹络，包括⼀个编码器和
⼀个解码器。性能最好的模型还通过注意机制将编码器和解码器连接起来。我们提出了⼀
个新的简单的⽹络结构 –Transformer ，它只基于注意⼒机制，完全不需要递归和卷积。
在两个机器翻译任务上的实验表明，这些模型在质量上更胜⼀筹，同时也更容易并⾏化，
需要的训练时间也⼤⼤减少。我们的模型在 WMT 2014 英德翻译任务中达到了 28.4 
BLEU ，⽐现有的最佳结果（包括合集）提⾼了 2 BLEU 以上。在 WMT 2014 英法翻译任
务中，我们的模型在 8 个 GPU 上训练了 3.5 天后，建⽴了新的单模型最先进的 BLEU 得分，
即 41.0 分，这只是⽂献中最佳模型的训练成本的⼀⼩部分。
2.GPT-3
GPT 家族与  BERT 模型都是知名的  NLP 预训练模型，都基于  Transformer 技术。 GPT
1 只有 12 个  Transformer 层，⽽到了  GPT3 ，则增加到  96 层。
论⽂标题：Language Models are Few-Shot Learners
论⽂链接：https://arxiv.org/pdf/2005.14165.pdf
摘要：最近的⼯作表明，在许多 NLP 任务和基准上，通过对⼤型⽂本语料库进⾏预训练，
然后对特定的任务进⾏微调，可以获得巨⼤的收益。虽然在结构上通常是任务⽆关的，但
这种⽅法仍然需要特定任务的微调数据集，包括⼏千或⼏万个例⼦。相⽐之下，⼈类通常
只需通过⼏个例⼦或简单的指令就能完成⼀项新的语⾔任务 – ⽽⽬前的 NLP 系统在很⼤程
度上仍难以做到这⼀点。在这⾥，我们展⽰了扩⼤语⾔模型的规模，⼤⼤改善了与任务⽆
关的、少量的性能，有时甚⾄达到了与之前最先进的微调⽅法的竞争⼒。具体来

### 🌐 載入 Wikipedia 文章（WikipediaLoader）

In [11]:
from langchain_community.document_loaders import WikipediaLoader

In [12]:
loader = WikipediaLoader(query="天壇", load_max_docs=3, lang="zh-TW")
docs = loader.load()
docs

[Document(metadata={'title': '天坛', 'summary': '天坛（满语：ᠠᠪᡣᠠᡞᠮᡠᡴ᠋ᡩᡝ᠋ᡥᡠᠨ；转写：abkai mukdehun），位于北京市东城区，是明清两朝皇帝祭天、祈穀和祈雨的场所。天坛始建于明成祖永乐十八年（公元1420年），原名“天地坛”，明嘉靖九年（公元1530年）在北京北郊另建祭祀地神的地坛，并改名为“天坛”。\n天坛佔地约273万平方米，是故宫面积的四倍，是现存中国古代规模最大、伦理等级最高的祭祀建筑群。1961年，天坛被国务院公布为第一批全国重点文物保护单位之一。1998年，“北京皇家祭坛—天坛”被列为世界文化遗产，亦為2024年7月27日所入選的另一項世界遺產「北京中轴线——中国理想都城秩序的杰作」的一部分。现时天坛公园还包括九坛八庙中的祈谷坛。\n天坛布局严谨，建筑结构独特，装饰瑰丽，巧妙地运用力学、声学和几何学等原理，具有较高的历史、科学和文化价值。', 'source': 'https://zh.wikipedia.org/wiki/%E5%A4%A9%E5%9D%9B'}, page_content='天坛（满语：ᠠᠪᡣᠠᡞᠮᡠᡴ᠋ᡩᡝ᠋ᡥᡠᠨ；转写：abkai mukdehun），位于北京市东城区，是明清两朝皇帝祭天、祈穀和祈雨的场所。天坛始建于明成祖永乐十八年（公元1420年），原名“天地坛”，明嘉靖九年（公元1530年）在北京北郊另建祭祀地神的地坛，并改名为“天坛”。\n天坛佔地约273万平方米，是故宫面积的四倍，是现存中国古代规模最大、伦理等级最高的祭祀建筑群。1961年，天坛被国务院公布为第一批全国重点文物保护单位之一。1998年，“北京皇家祭坛—天坛”被列为世界文化遗产，亦為2024年7月27日所入選的另一項世界遺產「北京中轴线——中国理想都城秩序的杰作」的一部分。现时天坛公园还包括九坛八庙中的祈谷坛。\n天坛布局严谨，建筑结构独特，装饰瑰丽，巧妙地运用力学、声学和几何学等原理，具有较高的历史、科学和文化价值。\n\n\n== 历史 ==\n\n北京天坛最初为明永乐十八年（1420年）仿南京大祀坛形制而建的天地坛，嘉靖九年（1530年）实行四郊分祀制度后，在北郊觅地另建地坛，原天地坛则专事祭天、祈穀和祈雨，并改名为天坛。清代基本沿袭明制，在乾隆年间曾进行过大规模的改扩建，但年门和皇乾

In [13]:
print(docs[0].page_content)

天坛（满语：ᠠᠪᡣᠠᡞᠮᡠᡴ᠋ᡩᡝ᠋ᡥᡠᠨ；转写：abkai mukdehun），位于北京市东城区，是明清两朝皇帝祭天、祈穀和祈雨的场所。天坛始建于明成祖永乐十八年（公元1420年），原名“天地坛”，明嘉靖九年（公元1530年）在北京北郊另建祭祀地神的地坛，并改名为“天坛”。
天坛佔地约273万平方米，是故宫面积的四倍，是现存中国古代规模最大、伦理等级最高的祭祀建筑群。1961年，天坛被国务院公布为第一批全国重点文物保护单位之一。1998年，“北京皇家祭坛—天坛”被列为世界文化遗产，亦為2024年7月27日所入選的另一項世界遺產「北京中轴线——中国理想都城秩序的杰作」的一部分。现时天坛公园还包括九坛八庙中的祈谷坛。
天坛布局严谨，建筑结构独特，装饰瑰丽，巧妙地运用力学、声学和几何学等原理，具有较高的历史、科学和文化价值。


== 历史 ==

北京天坛最初为明永乐十八年（1420年）仿南京大祀坛形制而建的天地坛，嘉靖九年（1530年）实行四郊分祀制度后，在北郊觅地另建地坛，原天地坛则专事祭天、祈穀和祈雨，并改名为天坛。清代基本沿袭明制，在乾隆年间曾进行过大规模的改扩建，但年门和皇乾殿是明代建筑而無改建除外。
明嘉靖十一年（1532年），在圜丘坛外泰元门东建起崇雩坛。
明嘉靖十七年（1538年），实行四郊分祀之后，大祀殿废而不用。嘉靖皇帝诏令拆大祀殿。
明嘉靖十九年（1540年），在大祀殿原址筹建大享殿。明嘉靖二十四年（1545年）8月，大享殿建成。
清乾隆十二年（1747年），拆除明嘉靖年间建造的崇雩坛。
清乾隆十六年（1751年），乾隆帝降旨改大享殿为祈年殿，大享门为祈年门。清乾隆十七年（1752年），修理圜丘、皇穹宇。祈年殿三覆檐上层青瓦、中层黄瓦、下层绿瓦及其祈年门、两庑、皇乾殿绿瓦均改为青色琉璃，其它仍照旧制盖覆绿瓦。
清光绪十五年（1889年），祈年殿毁于雷火。次年（1890年），祈年殿重建。  直至清光绪二十二年（1896年），祈年殿重建完工。
1900年八国联军進攻北京時，甚至还把司令部设在这里，並在圜丘坛上架设大炮，攻击正阳门和紫禁城，联军们将几乎所有的陈设和祭器都席卷而去。
1911年，辛亥革命爆发。1912年中华民国成立后，除了中華民國大總統袁世凯在1913年冬至祭天外，天坛不再进行任何祭祀活动，专用于为皇帝祭祀服务的天坛从此“任人游览”。19

### ✂️ Step 2：文件分段（TextSplitter）
> 長文需切割為適合嵌入與查詢的段落
* chunk_size：每段最大字元數。
* chunk_overlap：段落之間保留多少重疊內容，避免斷章取義。

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [15]:
loader=TextLoader("demo.txt",encoding="utf-8")
docs=loader.load()
docs

[Document(metadata={'source': 'demo.txt'}, page_content='凡爾賽宮（法語：Château de Versailles）位於法國巴黎西南郊外伊夫林省的省會凡爾賽，1682年由太陽王路易十四開始建造，分多階段完工，在此期間的凡爾賽宮超越了巴黎，成為了法國實際意義上的首都。\n\n凡爾賽宮有著寢宮、花園、美術收藏庫、辯論場、劇場、情報中心和政治會議室等諸多功能，是一座兼具觀賞性和實用性的宮殿，以金色、藍色和粉橘色為主基調，傾其所能表現「絢爛豪華的奢侈美」，這種風格被後世稱為「巴洛克」風格，不過在其後任法國國王路易十五的末期和整個路易十六時期，還加入了一些甜美風格「洛可可」裝飾。到了20世紀以後，法國因為進入共和政府，皇室被廢除，於是改建為博物館對公眾開放，一躍成為法國著名景點和文化遺產。[1]並在1979年被聯合國教科文組織列為世界文化遺產。[2]\n\n凡爾賽宮的建築風格因為極盡奢華，在17～18世紀的歐洲各國引發巨大轟動，神聖羅馬帝國（現奧地利和德國）、俄羅斯帝國、波蘭立陶宛聯邦和瑞典王國等歐洲大國的君主紛紛倣效，形成一股修建皇宮的跟風之潮。\n\n興建\n\n凡爾賽宮所在地區原來是一片森林和沼澤荒地。1624年，法國國王路易十三以1萬里弗爾的價格買下117法畝荒地，在這裡修建一座二層的紅磚樓房，用作狩獵行宮。[3][4]二樓有國王辦公室、寢室、接見室、藏衣室、隨從人員臥室等房間，一層為家具儲藏室和兵器庫。當時的行宮擁有26個房間，如今擁有2300個房間、67個樓梯和5210件家具[5][6]。\n\n1660年，法國國王路易十四參觀財政大臣富凱的沃子爵城堡[7]，為其房屋與花園的宏偉壯麗所折服，當時王室在巴黎郊外的行宮——聖日耳曼宮、萬塞訥宮、聖克盧宮等無一可以與其相比。路易十四憤怒他不盡職守之餘將富凱以「貪污」罪名投入巴士底獄，並命令沃子爵府邸的兩位主要設計師：園林家安德烈·勒諾特爾和建築家路易·勒沃為其設計新的行宮。[8][9][10]16至17世紀期間巴黎市民不斷發生暴動，更是在1648年至1653年，發生兩次規模巨大的投石黨叛亂，導致路易十四決定將王室宮廷（當時位於巴黎城內的羅浮宮和杜伊勒里宮）遷出混亂喧鬧的巴黎城。[11][12]經過考察和權衡，他決定以路易十三在凡爾賽的狩獵行宮為基礎建造新宮殿。為此

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=40,
    separators=["\n\n", "\n", "。", "！", "？", "，", "、", ""]
)

In [17]:
texts=text_splitter.split_documents(docs)
texts

[Document(metadata={'source': 'demo.txt'}, page_content='凡爾賽宮（法語：Château de Versailles）位於法國巴黎西南郊外伊夫林省的省會凡爾賽，1682年由太陽王路易十四開始建造，分多階段完工，在此期間的凡爾賽宮超越了巴黎，成為了法國實際意義上的首都。\n\n凡爾賽宮有著寢宮、花園、美術收藏庫、辯論場、劇場、情報中心和政治會議室等諸多功能，是一座兼具觀賞性和實用性的宮殿，以金色、藍色和粉橘色為主基調，傾其所能表現「絢爛豪華的奢侈美」，這種風格被後世稱為「巴洛克」風格，不過在其後任法國國王路易十五的末期和整個路易十六時期，還加入了一些甜美風格「洛可可」裝飾。到了20世紀以後，法國因為進入共和政府，皇室被廢除，於是改建為博物館對公眾開放，一躍成為法國著名景點和文化遺產。[1]並在1979年被聯合國教科文組織列為世界文化遺產。[2]\n\n凡爾賽宮的建築風格因為極盡奢華，在17～18世紀的歐洲各國引發巨大轟動，神聖羅馬帝國（現奧地利和德國）、俄羅斯帝國、波蘭立陶宛聯邦和瑞典王國等歐洲大國的君主紛紛倣效，形成一股修建皇宮的跟風之潮。\n\n興建'),
 Document(metadata={'source': 'demo.txt'}, page_content='興建\n\n凡爾賽宮所在地區原來是一片森林和沼澤荒地。1624年，法國國王路易十三以1萬里弗爾的價格買下117法畝荒地，在這裡修建一座二層的紅磚樓房，用作狩獵行宮。[3][4]二樓有國王辦公室、寢室、接見室、藏衣室、隨從人員臥室等房間，一層為家具儲藏室和兵器庫。當時的行宮擁有26個房間，如今擁有2300個房間、67個樓梯和5210件家具[5][6]。'),
 Document(metadata={'source': 'demo.txt'}, page_content='1660年，法國國王路易十四參觀財政大臣富凱的沃子爵城堡[7]，為其房屋與花園的宏偉壯麗所折服，當時王室在巴黎郊外的行宮——聖日耳曼宮、萬塞訥宮、聖克盧宮等無一可以與其相比。路易十四憤怒他不盡職守之餘將富凱以「貪污」罪名投入巴士底獄，並命令沃子爵府邸的兩位主要設計師：園林家安德烈·勒諾特爾和建築家路易·勒沃為其設計新的行宮。[8][9][10]16至17世紀期間巴黎市民不斷發生暴

In [18]:
print(texts[0].page_content)

凡爾賽宮（法語：Château de Versailles）位於法國巴黎西南郊外伊夫林省的省會凡爾賽，1682年由太陽王路易十四開始建造，分多階段完工，在此期間的凡爾賽宮超越了巴黎，成為了法國實際意義上的首都。

凡爾賽宮有著寢宮、花園、美術收藏庫、辯論場、劇場、情報中心和政治會議室等諸多功能，是一座兼具觀賞性和實用性的宮殿，以金色、藍色和粉橘色為主基調，傾其所能表現「絢爛豪華的奢侈美」，這種風格被後世稱為「巴洛克」風格，不過在其後任法國國王路易十五的末期和整個路易十六時期，還加入了一些甜美風格「洛可可」裝飾。到了20世紀以後，法國因為進入共和政府，皇室被廢除，於是改建為博物館對公眾開放，一躍成為法國著名景點和文化遺產。[1]並在1979年被聯合國教科文組織列為世界文化遺產。[2]

凡爾賽宮的建築風格因為極盡奢華，在17～18世紀的歐洲各國引發巨大轟動，神聖羅馬帝國（現奧地利和德國）、俄羅斯帝國、波蘭立陶宛聯邦和瑞典王國等歐洲大國的君主紛紛倣效，形成一股修建皇宮的跟風之潮。

興建


###🗂️ Step ３：文字嵌入（Embeddings）與建立向量資料庫（FAISS）

In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [24]:
loader = TextLoader("demo2.txt",encoding="utf-8")
docs = loader.load()

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=40,
    separators=["\n\n", "\n", "。", "！", "？", "，", "、", ""]
)


In [27]:
texts = text_splitter.split_documents(docs)


In [29]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

embeddings_model = OpenAIEmbeddings(api_key=api_key)

In [30]:
db = FAISS.from_documents(texts, embeddings_model)

### 🔍 Step 4：建立 Retriever 查詢器與整合 RetrievalQA 問答鏈

In [31]:
retriever = db.as_retriever()

In [32]:
retrieved_docs = retriever.invoke("請簡易介紹凡爾賽宮")
print(retrieved_docs[0].page_content)

凡爾賽宮橘園則位於宮殿南面，同樣也是勒沃於1663年所設計建造，1681年-1685年，橘園進行了全面重建並將其原有的規模擴大了一倍，當前該設施種植了許多如橘子樹、月桂樹、石榴、桃金孃等繁盛的植物，也是花園最熱門的景點之一。[76]

1679年末[77]，路易十四曾委託芒薩爾建造馬爾利城堡[78]，該建築位於凡爾賽宮邊緣的一處隱居處，距離宮殿約8公里（5英里）。城堡由一座主要住宅樓和十二個帕拉第奧式建築風格的亭子所組成，[79]馬爾利城堡在大革命後的1799年被國有化出售[80]，隨後在1805年遭到拆除，取而代之成為工業區的用地，[81]2009年6月1日，馬爾城堡遺址的土地被轉讓給凡爾賽宮、博物館和國家莊園的公共機構。[81]

燈籠小屋是一個狩獵小屋，以位於附近動物園頂部的燈籠命名，該建築由時任宮廷總督菲利普·路易·德·諾瓦耶於1787年建造。自1960年以來，它一直是總理府邸。[75]


In [33]:
retrieved_docs = retriever.invoke("凡爾賽宮在哪年被命名為法國歷史博物館")
print(retrieved_docs[0].page_content)

1889年，為紀念法國大革命100周年，法國政府（法蘭西第三共和國）把凡爾賽宮改造為公共博物館，裡面的展品全部供人展出、一般市民也可參觀。[36]1919年，第一次世界大戰所召開的巴黎和會在鏡廳召開，法國政府（法蘭西第三共和國）藉此羞辱戰敗德國、報德國統一時候的一箭之仇；但同時，被羞辱的德國也因此記恨法國，為二戰埋下伏筆，間接導致煽動德國愛國情緒的希特勒崛起。協約國於6月28日在鏡廳與德國簽訂《凡爾賽和約》。1920年，在大特里亞農宮簽訂製裁匈牙利王國的特里亞農和約。[37]1937年，凡爾賽宮作為歷史博物館對公眾開放。法國政府多次在宮中和花園中舉辦外事活動，召開國際會議，簽署國際條約。[38]


### 範例檔案 demo2.txt

In [34]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [35]:
loader = TextLoader("demo2.txt",encoding="utf-8")
docs = loader.load()

In [36]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=40,
    separators=["\n", "。", "！", "？", "，", "、", ""]
)

In [37]:
texts = text_splitter.split_documents(docs)

In [39]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
embeddings_model = OpenAIEmbeddings(api_key=api_key)

In [41]:
db = FAISS.from_documents(texts, embeddings_model)

In [42]:
retriever = db.as_retriever()

In [44]:
model = ChatOpenAI(model="gpt-3.5-turbo",api_key=api_key)

In [45]:
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history', output_key='answer')

<ipython-input-45-04112fe92479>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history', output_key='answer')


### 🔍背景與上下文
> LangChain 的 RetrievalQA 結構中，qa 是一個整合了：

* LLM（大型語言模型）
* Retriever（用來檢索向量知識庫）
* 問答 Chain（通常基於 Prompt 模板）

它可以讓你針對使用者問題，從「自建知識庫」中抓出相關段落，再讓 LLM 用這些內容來生成答案

—— 這就是典型的 RAG（檢索增強生成） 架構。

In [46]:
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory
)

In [47]:
question = "請簡易介紹凡爾賽宮"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='請簡易介紹凡爾賽宮', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮是法國的一座宮殿，由路易十三建立，後來被路易十四擴建成為一個宏偉的皇宮。宮殿被包圍著廣闊的園區，綿延達800公頃。園區內有宮殿、附屬建築、公園、花園等設施，長久以來一直是法國歷史的重要象徵。凡爾賽宮建築華麗，園區景觀優美，是重要的旅遊景點和文化遺產。', additional_kwargs={}, response_metadata={})],
 'question': '請簡易介紹凡爾賽宮',
 'answer': '凡爾賽宮是法國的一座宮殿，由路易十三建立，後來被路易十四擴建成為一個宏偉的皇宮。宮殿被包圍著廣闊的園區，綿延達800公頃。園區內有宮殿、附屬建築、公園、花園等設施，長久以來一直是法國歷史的重要象徵。凡爾賽宮建築華麗，園區景觀優美，是重要的旅遊景點和文化遺產。'}

In [48]:
question = "凡爾賽宮對應的拉丁文是什麼呢？"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='請簡易介紹凡爾賽宮', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮是法國的一座宮殿，由路易十三建立，後來被路易十四擴建成為一個宏偉的皇宮。宮殿被包圍著廣闊的園區，綿延達800公頃。園區內有宮殿、附屬建築、公園、花園等設施，長久以來一直是法國歷史的重要象徵。凡爾賽宮建築華麗，園區景觀優美，是重要的旅遊景點和文化遺產。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='凡爾賽宮對應的拉丁文是什麼呢？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮对应的拉丁文是"Château de Versailles"。', additional_kwargs={}, response_metadata={})],
 'question': '凡爾賽宮對應的拉丁文是什麼呢？',
 'answer': '凡爾賽宮对应的拉丁文是"Château de Versailles"。'}

In [49]:
qa = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

In [50]:
question = "請簡易介紹凡爾賽宮"
qa.invoke({"chat_history": memory, "question": question})

{'chat_history': [HumanMessage(content='請簡易介紹凡爾賽宮', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮是法國的一座宮殿，由路易十三建立，後來被路易十四擴建成為一個宏偉的皇宮。宮殿被包圍著廣闊的園區，綿延達800公頃。園區內有宮殿、附屬建築、公園、花園等設施，長久以來一直是法國歷史的重要象徵。凡爾賽宮建築華麗，園區景觀優美，是重要的旅遊景點和文化遺產。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='凡爾賽宮對應的拉丁文是什麼呢？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮对应的拉丁文是"Château de Versailles"。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='請簡易介紹凡爾賽宮', additional_kwargs={}, response_metadata={}),
  AIMessage(content='凡爾賽宮是位於法國的一座宏偉的宮殿，始建於17世紀，是法國君主的權力中心。宮殿包括許多華麗的房間和廳堂，以及壯麗的花園和噴泉。凡爾賽宮在設計上極具巴洛克風格，展示了法國宮廷生活及藝術領域的瑰麗風貌。除了作為藝術珍品的展示場所外，凡爾賽宮還扮演著重要的歷史角色，影響著歐洲其他宮殿的建築和設計。自1979年起，凡爾賽宮被列入聯合國教科文組織的世界遺產名錄，成為重要的文化遺產之一。', additional_kwargs={}, response_metadata={})],
 'question': '請簡易介紹凡爾賽宮',
 'answer': '凡爾賽宮是位於法國的一座宏偉的宮殿，始建於17世紀，是法國君主的權力中心。宮殿包括許多華麗的房間和廳堂，以及壯麗的花園和噴泉。凡爾賽宮在設計上極具巴洛克風格，展示了法國宮廷生活及藝術領域的瑰麗風貌。除了作為藝術珍品的展示場所外，凡爾賽宮還扮演著重要的歷史角色，影響著歐洲其

In [ ]:
print()